In [1]:
# Instructiuni:
# iau din train le dau la antrenat 
# dupa verific acuratetea pe validare
# dupa fac un predict pe cele de test si scriu intr-un csv imaginea si clasa lor si pe ala il incarc pe kaggle

In [2]:
import numpy as np
import matplotlib.pyplot as plt

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import RMSprop

import tensorflow.compat as tf
#import tensorflow.compat.v1 as tf 
#tf.disable_v2_behavior()

from tensorflow import keras
from tensorflow.keras import layers

import csv

In [3]:
# citire imagini de antrenare plus etichete de antrenare
f = open ("train.txt")
train_images = []
train_labels = []
for line in enumerate(open("train.txt", "r")):
    v = line[1].split(',')
    image_name = 'train/'+v[0]
    label = int(v[1])
    train_labels.append(label)
    image = plt.imread(image_name)
    image=image.reshape(-1) # din 2D => 1D
    train_images.append(image)
# transformare în nparray
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [4]:
# citire imagini de testare 
f = open ("test.txt")
test_images = []
name_test_images =[]
for line in enumerate(open("test.txt", "r")):
    image_name = 'test/'+ line[1][:len(line[1])-1]
    name_test_images.append(line[1][:len(line[1])-1])
    image = plt.imread(image_name)
    image=image.reshape(-1) # din 2D => 1D
    test_images.append(image)
# transformare în nparray
test_images = np.array(test_images)

In [5]:
# citire imagini de validare plus etichete de validare
f = open ("validation.txt")
validation_images = []
validation_labels = []
for line in enumerate(open("validation.txt", "r")):
    v = line[1].split(',')
    image_name = 'validation/'+v[0]
    label = int(v[1])
    validation_labels.append(label)
    image = plt.imread(image_name)
    image=image.reshape(-1) # din 2D => 1D
    validation_images.append(image)
# transformare în nparray
validation_images = np.array(validation_images)
validation_labels = np.array(validation_labels)

In [6]:
nb_classes = 9 #9 categorii notate de la 0 la 8

x_train=train_images.astype('float32')
x_val=validation_images.astype('float32')
test_images=test_images.astype('float32')

y_val=np_utils.to_categorical(validation_labels, nb_classes) 
y_train=np_utils.to_categorical(train_labels, nb_classes) 

In [7]:
# Marire esantion de date
# creez un esantion separat
# prin aplicarea de transformari aleatorii imaginilor de antrenament 
# Scop: evitarea overfitting-ului & expunerea modelului la mai multe cazuri de studiu  
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal"), #rasucire orizontala
        layers.experimental.preprocessing.RandomRotation(0.1), #rotatie aleatorie a imaginii
    ]
)
# creare model de baza
base_model = keras.applications.Xception(
    weights="imagenet",  # incarcare date preantrenate din ImageNet
    input_shape=(150, 150, 3), #dimensiuni ImageNet
    include_top=False,
)  

# "Dezactivare" model de baza (Freezing) 
base_model.trainable = False

# Craere model nou 
inputs = keras.Input(shape=(32, 32, 3)) 
x = data_augmentation(inputs)  # Aplicare transformari aleatorii 

# Adaug un layer de normalizare pe intervalele [0,255] & [-1,1] => datele preantrenate au nevoie de normalizare
norm_layer = keras.layers.experimental.preprocessing.Normalization()
mean = np.array([127.5] * 3)
var = mean ** 2
# [-1,1]
x = norm_layer(x)
norm_layer.set_weights([mean, var])

# "Batch normalization" -> model mai rapid si stabil, normalizare layer de intrare (prin recentrare si redimensionare)
# Modelul de baza contine layere batch-norm 
# Pastrare model in modul de inferenta (inference mode) -> pregatirea pentru fine-tuning  
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)

# Pentru regularizare adaug un Dropout layer => Scop: reducere overfitting
x = keras.layers.Dropout(0.2)(x)  

outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

#model.summary()

In [8]:
model = Sequential()
model.add(Dense(100, input_shape =(x_train.shape[1],)))
model.add(Activation ('relu'))
model.add(Dense(200))
model.add(Activation ('sigmoid'))
model.add(Dense(nb_classes))
model.add(Activation ('softmax'))

model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001, rho=0.9, epsilon=1e-6, decay=1e-6), metrics=['accuracy'])
history= model.fit (x_train, y_train, batch_size=200, epochs=100, verbose=2)
loss_and_metrics = model.evaluate(x_val, y_val)
print(loss_and_metrics)

Epoch 1/100
151/151 - 3s - loss: 0.3038 - accuracy: 0.3671
Epoch 2/100
151/151 - 1s - loss: 0.2511 - accuracy: 0.4989
Epoch 3/100
151/151 - 1s - loss: 0.2292 - accuracy: 0.5498
Epoch 4/100
151/151 - 1s - loss: 0.2146 - accuracy: 0.5856
Epoch 5/100
151/151 - 1s - loss: 0.2043 - accuracy: 0.6082
Epoch 6/100
151/151 - 1s - loss: 0.1971 - accuracy: 0.6240
Epoch 7/100
151/151 - 1s - loss: 0.1903 - accuracy: 0.6387
Epoch 8/100
151/151 - 1s - loss: 0.1846 - accuracy: 0.6533
Epoch 9/100
151/151 - 1s - loss: 0.1807 - accuracy: 0.6592
Epoch 10/100
151/151 - 1s - loss: 0.1756 - accuracy: 0.6717
Epoch 11/100
151/151 - 1s - loss: 0.1736 - accuracy: 0.6751
Epoch 12/100
151/151 - 1s - loss: 0.1706 - accuracy: 0.6844
Epoch 13/100
151/151 - 1s - loss: 0.1665 - accuracy: 0.6884
Epoch 14/100
151/151 - 1s - loss: 0.1646 - accuracy: 0.6942
Epoch 15/100
151/151 - 1s - loss: 0.1605 - accuracy: 0.7013
Epoch 16/100
151/151 - 1s - loss: 0.1581 - accuracy: 0.7084
Epoch 17/100
151/151 - 1s - loss: 0.1556 - accura

In [9]:
# Antrenare model principal
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)


In [10]:
# "Fine tunning"
# "Activare" (Unfreeze) model de baza aflat in "inference mode"

base_model.trainable = True
model.summary()


model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # rata de invatare mica
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

history= model.fit (x_train, y_train, batch_size=200, epochs=20, verbose=2)
loss_and_metrics = model.evaluate(x_val, y_val)
print(loss_and_metrics)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               102500    
_________________________________________________________________
activation (Activation)      (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               20200     
_________________________________________________________________
activation_1 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 9)                 1809      
_________________________________________________________________
activation_2 (Activation)    (None, 9)                 0         
Total params: 124,509
Trainable params: 124,509
Non-trainable params: 0
________________________________________________

In [15]:
predict =model.predict_classes(test_images).astype("int32")
f = open("sub2.csv", "w", newline="")
csv_file= csv.writer(f, delimiter=",")
csv_file.writerow(['id', 'label'])
for i in range (predict.shape[0]):
    print(name_test_images[i], ",", predict[i], sep="")
    csv_file.writerow([name_test_images[i], predict[i]])
f.close()

[5 0 4 5 4 0 3 7 7 7 1 5 5 1 4 7 1 4 1 0 0 0 5 8 4 7 3 5 6 2 2 8 0 1 3 7 7
 4 6 0 3 6 0 0 8 5 4 3 3 3 0 3 0 1 1 7 7 7 4 1 8 4 6 1 3 5 4 4 7 1 3 1 0 1
 6 4 0 0 7 4 7 3 8 3 1 5 5 5 7 2 4 8 5 3 6 8 1 1 0 2]
035001.png,5
035002.png,0
035003.png,4
035004.png,5
035005.png,4
035006.png,0
035007.png,3
035008.png,7
035009.png,7
035010.png,7
035011.png,1
035012.png,5
035013.png,5
035014.png,1
035015.png,4
035016.png,7
035017.png,1
035018.png,4
035019.png,1
035020.png,0
035021.png,0
035022.png,0
035023.png,5
035024.png,8
035025.png,4
035026.png,7
035027.png,3
035028.png,5
035029.png,6
035030.png,2
035031.png,2
035032.png,8
035033.png,0
035034.png,1
035035.png,3
035036.png,7
035037.png,7
035038.png,4
035039.png,6
035040.png,0
035041.png,3
035042.png,6
035043.png,0
035044.png,0
035045.png,8
035046.png,5
035047.png,4
035048.png,3
035049.png,3
035050.png,3
035051.png,0
035052.png,3
035053.png,0
035054.png,1
035055.png,1
035056.png,7
035057.png,7
035058.png,7
035059.png,4
035060.png,1
035061.png,8
035

035797.png,3
035798.png,2
035799.png,5
035800.png,0
035801.png,5
035802.png,8
035803.png,8
035804.png,5
035805.png,2
035806.png,8
035807.png,1
035808.png,3
035809.png,6
035810.png,5
035811.png,1
035812.png,6
035813.png,2
035814.png,7
035815.png,5
035816.png,1
035817.png,5
035818.png,0
035819.png,4
035820.png,5
035821.png,0
035822.png,8
035823.png,6
035824.png,6
035825.png,7
035826.png,6
035827.png,5
035828.png,6
035829.png,6
035830.png,0
035831.png,3
035832.png,1
035833.png,8
035834.png,3
035835.png,8
035836.png,8
035837.png,6
035838.png,1
035839.png,4
035840.png,7
035841.png,6
035842.png,4
035843.png,5
035844.png,7
035845.png,1
035846.png,7
035847.png,1
035848.png,8
035849.png,5
035850.png,3
035851.png,1
035852.png,1
035853.png,1
035854.png,7
035855.png,8
035856.png,3
035857.png,3
035858.png,5
035859.png,2
035860.png,6
035861.png,5
035862.png,7
035863.png,3
035864.png,8
035865.png,2
035866.png,4
035867.png,8
035868.png,8
035869.png,0
035870.png,5
035871.png,0
035872.png,6
035873.png,0

036631.png,8
036632.png,4
036633.png,6
036634.png,0
036635.png,0
036636.png,4
036637.png,2
036638.png,4
036639.png,5
036640.png,4
036641.png,8
036642.png,4
036643.png,5
036644.png,2
036645.png,5
036646.png,6
036647.png,5
036648.png,5
036649.png,4
036650.png,6
036651.png,6
036652.png,3
036653.png,5
036654.png,3
036655.png,6
036656.png,8
036657.png,8
036658.png,0
036659.png,8
036660.png,6
036661.png,1
036662.png,0
036663.png,7
036664.png,3
036665.png,3
036666.png,5
036667.png,1
036668.png,3
036669.png,7
036670.png,1
036671.png,6
036672.png,0
036673.png,3
036674.png,2
036675.png,6
036676.png,2
036677.png,1
036678.png,8
036679.png,7
036680.png,0
036681.png,4
036682.png,6
036683.png,5
036684.png,0
036685.png,6
036686.png,7
036687.png,2
036688.png,6
036689.png,3
036690.png,1
036691.png,5
036692.png,4
036693.png,4
036694.png,6
036695.png,8
036696.png,0
036697.png,6
036698.png,1
036699.png,0
036700.png,6
036701.png,2
036702.png,4
036703.png,6
036704.png,2
036705.png,5
036706.png,8
036707.png,8

037494.png,3
037495.png,3
037496.png,7
037497.png,6
037498.png,2
037499.png,7
037500.png,3
037501.png,3
037502.png,1
037503.png,1
037504.png,0
037505.png,0
037506.png,5
037507.png,4
037508.png,4
037509.png,1
037510.png,7
037511.png,0
037512.png,5
037513.png,6
037514.png,5
037515.png,2
037516.png,0
037517.png,3
037518.png,3
037519.png,7
037520.png,3
037521.png,8
037522.png,5
037523.png,4
037524.png,7
037525.png,2
037526.png,4
037527.png,3
037528.png,3
037529.png,2
037530.png,0
037531.png,3
037532.png,4
037533.png,0
037534.png,8
037535.png,3
037536.png,1
037537.png,5
037538.png,3
037539.png,2
037540.png,1
037541.png,8
037542.png,7
037543.png,4
037544.png,2
037545.png,2
037546.png,1
037547.png,7
037548.png,1
037549.png,3
037550.png,4
037551.png,1
037552.png,1
037553.png,0
037554.png,6
037555.png,7
037556.png,2
037557.png,5
037558.png,7
037559.png,0
037560.png,7
037561.png,6
037562.png,4
037563.png,1
037564.png,4
037565.png,2
037566.png,2
037567.png,7
037568.png,1
037569.png,4
037570.png,5

038297.png,6
038298.png,6
038299.png,1
038300.png,2
038301.png,1
038302.png,1
038303.png,2
038304.png,4
038305.png,7
038306.png,3
038307.png,5
038308.png,0
038309.png,2
038310.png,5
038311.png,2
038312.png,5
038313.png,5
038314.png,4
038315.png,6
038316.png,6
038317.png,8
038318.png,4
038319.png,4
038320.png,1
038321.png,2
038322.png,2
038323.png,6
038324.png,5
038325.png,6
038326.png,4
038327.png,3
038328.png,8
038329.png,3
038330.png,4
038331.png,6
038332.png,0
038333.png,8
038334.png,8
038335.png,6
038336.png,6
038337.png,7
038338.png,7
038339.png,2
038340.png,5
038341.png,5
038342.png,5
038343.png,2
038344.png,3
038345.png,6
038346.png,2
038347.png,7
038348.png,5
038349.png,2
038350.png,8
038351.png,7
038352.png,6
038353.png,6
038354.png,3
038355.png,8
038356.png,3
038357.png,8
038358.png,4
038359.png,5
038360.png,4
038361.png,6
038362.png,7
038363.png,2
038364.png,2
038365.png,6
038366.png,1
038367.png,2
038368.png,1
038369.png,6
038370.png,6
038371.png,5
038372.png,0
038373.png,7

039130.png,4
039131.png,4
039132.png,4
039133.png,1
039134.png,7
039135.png,7
039136.png,4
039137.png,0
039138.png,4
039139.png,5
039140.png,2
039141.png,5
039142.png,5
039143.png,4
039144.png,6
039145.png,2
039146.png,2
039147.png,1
039148.png,8
039149.png,7
039150.png,5
039151.png,6
039152.png,6
039153.png,4
039154.png,6
039155.png,3
039156.png,2
039157.png,1
039158.png,7
039159.png,7
039160.png,4
039161.png,2
039162.png,6
039163.png,0
039164.png,5
039165.png,5
039166.png,4
039167.png,1
039168.png,0
039169.png,7
039170.png,6
039171.png,0
039172.png,4
039173.png,5
039174.png,4
039175.png,2
039176.png,6
039177.png,3
039178.png,5
039179.png,5
039180.png,7
039181.png,5
039182.png,8
039183.png,8
039184.png,1
039185.png,6
039186.png,8
039187.png,3
039188.png,3
039189.png,8
039190.png,6
039191.png,1
039192.png,1
039193.png,3
039194.png,1
039195.png,2
039196.png,3
039197.png,7
039198.png,3
039199.png,4
039200.png,0
039201.png,6
039202.png,4
039203.png,5
039204.png,5
039205.png,7
039206.png,8

039963.png,8
039964.png,2
039965.png,0
039966.png,4
039967.png,1
039968.png,0
039969.png,8
039970.png,1
039971.png,8
039972.png,3
039973.png,5
039974.png,6
039975.png,4
039976.png,5
039977.png,6
039978.png,2
039979.png,1
039980.png,4
039981.png,1
039982.png,8
039983.png,2
039984.png,4
039985.png,4
039986.png,0
039987.png,3
039988.png,5
039989.png,2
039990.png,3
039991.png,4
039992.png,5
039993.png,1
039994.png,5
039995.png,0
039996.png,6
039997.png,4
039998.png,1
039999.png,7
040000.png,4
